<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_07/blob/main/validate_DPR_TREC_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import os

In [47]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [48]:
%%shell
pip install pyserini -q
git clone --recurse-submodules https://github.com/castorini/pyserini.git
cd pyserini
cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
cd tools/eval/ndeval && make && cd ../../..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00
Cloning into 'pyserini'...
remote: Enumerating objects: 7501, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 7501 (delta 371), reused 380 (delta 241), pack-reused 6954
Receiving objects: 100% (7501/7501), 4.41 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (5416/5416), done.
Updating f

In [1]:
!pip install transformers -q
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pickle
import numpy as np

from google.colab import drive

import torch

from scipy import stats

from datetime import datetime

from transformers import (AutoTokenizer, 
                          AutoModel,
                          BatchEncoding
)

from tqdm.auto import tqdm

import time

import faiss

In [3]:
WORKING_FOLDER="drive/MyDrive/unicamp/ia368v_dd/aula_07"

TREC_COVID_MERGED_FILE="trec_covid_merged_data.tsv"
TREC_COVID_QRELS="trec_covid_qrels.tsv"

TRAIN_OUTPUT_FOLDER="./trained_model"

ENCODED_DATA_FILE="trec_covid_encoded_data.pkl"

In [4]:
PYSERINI_TEST_RUN_DPR_FILENAME_FORMAT="run.trec-covid_DPR_{}.txt"

In [5]:
BEST_MODELS_CHECKPOINT="checkpoint_20230414_170604_0.0772"

MODEL_NAME='microsoft/MiniLM-L12-H384-uncased'

MODEL_MAX_INPUT_LENGTH=512

In [6]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
os.chdir(WORKING_FOLDER)

In [8]:
trec_covid_merged_df = pd.read_csv(TREC_COVID_MERGED_FILE, sep='\t')

## Explore TREC COVID merged data

This dataframe contains the merge of the TREC COVID qrels and the document corpus. Hence, it only lists the documents which are indeed mentioned in the qrels.

In [ ]:
trec_covid_merged_df

,query-id,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
0,1,005b2j4b,2,what is the origin of COVID-19,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'coronavirus origin', 'narrative': ""...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
1,16,005b2j4b,0,how long does coronavirus remain stable on su...,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,{'query': 'how long does coronavirus survive o...,{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
2,32,005b2j4b,0,"Does SARS-CoV-2 have any subtypes, and if so w...",Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'coronavirus subtypes', 'narrative':...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
3,37,005b2j4b,0,What is the result of phylogenetic analysis of...,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'SARS-CoV-2 phylogenetic analysis', ...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
4,1,00fmeepz,1,what is the origin of COVID-19,Comprehensive overview of COVID-19 based on cu...,"In December 2019, twenty-seven pneumonia patie...","{'query': 'coronavirus origin', 'narrative': ""...","{'url': '', 'pubmed_id': ''}"
...,...,...,...,...,...,...,...,...
66331,50,zn10rnrm,1,what is known about an mRNA vaccine for the SA...,Characterization of RNA in Saliva,Background: We have previously shown that huma...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://www.ncbi.nlm.nih.gov/pmc/arti...
66332,50,zstmdt4n,0,what is known about an mRNA vaccine for the SA...,Coordinate induction of IFN-α and -γ by SARS-C...,Abstract Background: Severe acute respiratory ...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://api.elsevier.com/content/arti...
66333,50,zth8ffy3,0,what is known about an mRNA vaccine for the SA...,Vasculopathy and Coagulopathy Associated with ...,The emergence of severe acute respiratory synd...,"{'query': 'mRNA vaccine coronavirus', 'narrati...","{'url': '', 'pubmed_id': ''}"
66334,50,zv4nbz9p,2,what is known about an mRNA vaccine for the SA...,"Emerging Technologies for Use in the Study, Di...",INTRODUCTION: The COVID-19 pandemic has caused...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://doi.org/10.1007/s12195-020-00...


#### Check the number of documents

In [ ]:
trec_covid_merged_df.drop_duplicates(['corpus-id'])

,query-id,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
0,1,005b2j4b,2,what is the origin of COVID-19,Monophyletic Relationship between Severe Acute...,Although primary genomic analysis has revealed...,"{'query': 'coronavirus origin', 'narrative': ""...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/1...
4,1,00fmeepz,1,what is the origin of COVID-19,Comprehensive overview of COVID-19 based on cu...,"In December 2019, twenty-seven pneumonia patie...","{'query': 'coronavirus origin', 'narrative': ""...","{'url': '', 'pubmed_id': ''}"
8,1,g7dhmyyo,2,what is the origin of COVID-19,"The SARS, MERS and novel coronavirus (COVID-19...",OBJECTIVES: To provide an overview of the thre...,"{'query': 'coronavirus origin', 'narrative': ""...","{'url': '', 'pubmed_id': ''}"
17,1,0194oljo,1,what is the origin of COVID-19,Evidence for zoonotic origins of Middle East r...,Middle East respiratory syndrome (MERS) is an ...,"{'query': 'coronavirus origin', 'narrative': ""...",{'url': 'https://www.ncbi.nlm.nih.gov/pubmed/2...
18,1,021q9884,1,what is the origin of COVID-19,Deadly virus effortlessly hops species,Genetic engineering helps reveal origin of dea...,"{'query': 'coronavirus origin', 'narrative': ""...",{'url': 'https://www.ncbi.nlm.nih.gov/pmc/arti...
...,...,...,...,...,...,...,...,...
66331,50,zn10rnrm,1,what is known about an mRNA vaccine for the SA...,Characterization of RNA in Saliva,Background: We have previously shown that huma...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://www.ncbi.nlm.nih.gov/pmc/arti...
66332,50,zstmdt4n,0,what is known about an mRNA vaccine for the SA...,Coordinate induction of IFN-α and -γ by SARS-C...,Abstract Background: Severe acute respiratory ...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://api.elsevier.com/content/arti...
66333,50,zth8ffy3,0,what is known about an mRNA vaccine for the SA...,Vasculopathy and Coagulopathy Associated with ...,The emergence of severe acute respiratory synd...,"{'query': 'mRNA vaccine coronavirus', 'narrati...","{'url': '', 'pubmed_id': ''}"
66334,50,zv4nbz9p,2,what is known about an mRNA vaccine for the SA...,"Emerging Technologies for Use in the Study, Di...",INTRODUCTION: The COVID-19 pandemic has caused...,"{'query': 'mRNA vaccine coronavirus', 'narrati...",{'url': 'https://doi.org/10.1007/s12195-020-00...


#### Check the number of documents referred by each question

In [ ]:
trec_covid_merged_df.groupby('query-id').count()

,corpus-id,score,query-text,corpus-title,corpus-text,query-metadata,corpus-metadata
query-id,,,,,,,
1,1565,1565,1565,1565,1354,1565,1565
2,1251,1251,1251,1251,1091,1251,1251
3,1649,1649,1649,1649,1457,1649,1649
4,1793,1793,1793,1793,1511,1793,1793
5,1643,1643,1643,1643,1479,1643,1643
6,1562,1562,1562,1562,1372,1562,1562
7,1331,1331,1331,1331,1156,1331,1331
8,1837,1837,1837,1837,1607,1837,1837
9,1606,1606,1606,1606,1327,1606,1606


#### Check the available document scores

In [ ]:
trec_covid_merged_df['score'].unique()

array([ 2,  0,  1, -1])

## Create dense representations for the documents and requires

In [9]:
class TextToEncodeDataset(torch.utils.data.Dataset):

    def __init__(self, texts_list, tokenizer, max_length=None):

        self.max_length = max_length

        self.tokenized_texts = tokenizer(texts_list, 
                                         truncation=True, 
                                         return_overflowing_tokens=True, 
                                         max_length=max_length, 
                                         return_length=True)
        
        self.original_length = len(texts_list)
        self.length_stats = stats.describe(self.tokenized_texts['length'])

        print("Text tokens size stats:\n{}\n".format(self.length_stats))

        if (max_length is not None) and 'overflow_to_sample_mapping' in self.tokenized_texts:
            if self.original_length < len(self.tokenized_texts['overflow_to_sample_mapping']):
                print("Added {} overflowing texts...".format(len(self.tokenized_texts['overflow_to_sample_mapping']) - self.original_length))


    def __len__(self):
        return len(self.tokenized_texts['input_ids'])


    def __getitem__(self, index):
        return {'input_ids': self.tokenized_texts['input_ids'][index],
                'attention_mask': self.tokenized_texts['attention_mask'][index]}

    def get_original_index(self, tokenized_documents_indexes):
        if 'overflow_to_sample_mapping' in self.tokenized_texts:
            return np.array(self.tokenized_texts['overflow_to_sample_mapping'])[tokenized_documents_indexes]

In [10]:
class DPRCollator(object):
    def __init__(self, tokenizer=None):
        self.tokenizer = tokenizer


    def __call__(self, batch):
        padded_batch = self.tokenizer.pad(batch, return_tensors='pt')

        return BatchEncoding(padded_batch)

In [11]:
def encode(device,
           which_model, 
           which_dataloader):
    
    encoded_text = []

    which_model.eval()

    with torch.no_grad():
        for batch in tqdm(which_dataloader, mininterval=0.5, desc="Encode", disable=False):
            
            encoded_outputs = which_model(**batch.to(device))

            encoded_cls = encoded_outputs.last_hidden_state[:, 0, :]

            encoded_text.append(encoded_cls.cpu().numpy())

    return np.concatenate(encoded_text, axis=0)

### Instantiate the required elements

In [12]:
batch_size = 256

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [15]:
passages_model = AutoModel.from_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, BEST_MODELS_CHECKPOINT, "_passages")).to(device)
topics_model = AutoModel.from_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, BEST_MODELS_CHECKPOINT, "_topics")).to(device)

In [16]:
valid_queries_df = trec_covid_merged_df[['query-id', 'query-text']].drop_duplicates().sort_values('query-id').reset_index(drop=True)

In [17]:
trec_covid_merged_df['corpus-text'] = trec_covid_merged_df['corpus-text'].fillna("")

In [18]:
trec_covid_merged_df['corpus-title-text'] = trec_covid_merged_df['corpus-title'] + trec_covid_merged_df['corpus-text']

In [19]:
valid_corpus_df = trec_covid_merged_df.drop_duplicates(['corpus-id']).reset_index(drop=True)

In [20]:
queries_to_encode = TextToEncodeDataset(valid_queries_df['query-text'].tolist(), tokenizer, MODEL_MAX_INPUT_LENGTH)

Text tokens size stats:
DescribeResult(nobs=50, minmax=(10, 36), mean=18.04, variance=30.814693877551026, skewness=0.9979912345213009, kurtosis=1.090310501135514)



In [21]:
start_time = time.time()

corpus_to_encode = TextToEncodeDataset(valid_corpus_df['corpus-title-text'].tolist(), tokenizer, MODEL_MAX_INPUT_LENGTH)

print("Time to tokenize the corpus: {}".format(time.time() - start_time))

Text tokens size stats:
DescribeResult(nobs=38486, minmax=(3, 512), mean=259.42464792392036, variance=24997.615460022855, skewness=-0.08536849715563527, kurtosis=-1.1039927293791474)

Added 3006 overflowing texts...


In [23]:
encode_queries_dataloader = torch.utils.data.DataLoader(queries_to_encode, 
                                                        batch_size=batch_size, 
                                                        shuffle=False, 
                                                        collate_fn=DPRCollator(tokenizer))

encode_corpus_dataloader = torch.utils.data.DataLoader(corpus_to_encode, 
                                                       batch_size=batch_size, 
                                                       shuffle=False, 
                                                       collate_fn=DPRCollator(tokenizer))

In [24]:
encoded_queries = encode(device, topics_model, encode_queries_dataloader)

Encode:   0%|          | 0/1 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
start_time = time.time()

encoded_corpus = encode(device, passages_model, encode_corpus_dataloader)

print("Time to encode the corpus: {}".format(time.time() - start_time))

Encode:   0%|          | 0/151 [00:00<?, ?it/s]

Time to encode the corpus: 502.79755687713623


In [27]:
with open(ENCODED_DATA_FILE, 'wb') as outputFile:
    pickle.dump({'encoded_queries': encoded_queries,
                 'encoded_corpus': encoded_corpus,
                 'queries_dataset': queries_to_encode,
                 'corpus_dataset': corpus_to_encode}, outputFile, pickle.HIGHEST_PROTOCOL)

## Now, create an search index

In [28]:
corpus_index = faiss.IndexFlatIP(384)

In [29]:
start_time = time.time()

corpus_index.add(encoded_corpus)

print("FAISS index creation time: {}".format(time.time() - start_time))

FAISS index creation time: 0.12183022499084473


## Search the queries in the index

In [40]:
search_results = corpus_index.search(encoded_queries, 1000)

In [41]:
search_results[0].shape

(50, 1000)

### Save the results in the TREC file format

In [42]:
TREC_RESULT_LINE_FORMAT="{}\tQ0\t{}\t{}\t{}\tminiLM_DPR\n"

In [58]:
test_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [59]:
run_filename = PYSERINI_TEST_RUN_DPR_FILENAME_FORMAT.format(test_timestamp)

In [60]:
with open(run_filename, 'w') as outputFile:
    for query_index, query_scores in enumerate(search_results[0]):
        document_descending_order = np.argsort(query_scores)[::-1]
        tokenized_documents_ordered_indexes = search_results[1][query_index][document_descending_order]

        original_documents_ordered_indexes = corpus_to_encode.get_original_index(tokenized_documents_ordered_indexes)

        included_docs = set()

        for i, document_index in enumerate(original_documents_ordered_indexes):
            if document_index not in included_docs:
                included_docs.add(document_index)

                outputFile.write(TREC_RESULT_LINE_FORMAT.format(valid_queries_df.iloc[query_index]['query-id'], 
                                                                valid_corpus_df.iloc[document_index]['corpus-id'], 
                                                                i + 1,
                                                                query_scores[document_descending_order[i]]))
            else:
                print("Ignoring document={} as it is already in the answers set for query={}".format(document_index, query_index))

Ignoring document=8958 as it is already in the answers set for query=0
Ignoring document=11446 as it is already in the answers set for query=0
Ignoring document=6710 as it is already in the answers set for query=0
Ignoring document=12048 as it is already in the answers set for query=1
Ignoring document=28788 as it is already in the answers set for query=1
Ignoring document=22816 as it is already in the answers set for query=2
Ignoring document=22809 as it is already in the answers set for query=2
Ignoring document=1531 as it is already in the answers set for query=2
Ignoring document=22809 as it is already in the answers set for query=2
Ignoring document=22816 as it is already in the answers set for query=2
Ignoring document=1034 as it is already in the answers set for query=2
Ignoring document=2497 as it is already in the answers set for query=2
Ignoring document=541 as it is already in the answers set for query=2
Ignoring document=4131 as it is already in the answers set for query=3


## Now run the metric

In [61]:
!pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank.100 \
    {TREC_COVID_QRELS} {run_filename}

map                   	all	0.0773
recip_rank            	all	0.5904
recall_1000           	all	0.2512
ndcg_cut_10           	all	0.3803
